In [25]:
import pandas as pd
import numpy as np

In [26]:
events = pd.read_csv("holidays_events.csv")
oil = pd.read_csv("oil.csv")
stores = pd.read_csv("stores.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
transactions = pd.read_csv("transactions.csv")

In [27]:
oil["dcoilwtico"].bfill(inplace=True)
oil.rename(columns={"dcoilwtico": "oil_price"}, inplace=True)
oil = oil.fillna(method="bfill")

C:\Users\Admin\AppData\Local\Temp\ipykernel_11388\285669028.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil = oil.fillna(method="bfill")


In [28]:
train = train.merge(oil, how="left", on="date")

In [29]:
events = events.drop("description", axis=1)

In [30]:
events = pd.get_dummies(events, columns=['type', 'locale', 'locale_name'], drop_first=True)
events = events.groupby("date").max().reset_index()

In [31]:
train = train.merge(events, on='date', how='left')
train = train.merge(stores, on="store_nbr", how="inner")

In [32]:
train = pd.get_dummies(train, columns=['city', 'state'], drop_first=True)
train.replace({True: 1, False: 0}, inplace=True)

In [33]:
train = train.merge(transactions, on=['store_nbr', 'date'], how='left')

In [34]:
train['oil_price'].fillna(method='ffill', inplace=True)
train.fillna(0, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11388\718790895.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['oil_price'].fillna(method='ffill', inplace=True)


In [35]:
train = train.drop(columns=['id', 'store_nbr', 'cluster', 'type', 'onpromotion'])
train['date'] = pd.to_datetime(train['date'])
train['day_of_year'] = train['date'].dt.dayofyear
train['day_of_week'] = train['date'].dt.dayofweek
train.drop(columns=['date'], axis=1, inplace=True)

In [36]:
train = pd.get_dummies(train, columns=['family'], drop_first=True)
train

,sales,oil_price,transferred,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,locale_National,locale_Regional,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,0.000,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,0.000,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,0.000,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,0.000,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,0.000,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,438.133,47.57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
3000884,154.553,47.57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,True,False,False,False
3000885,2419.729,47.57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False
3000886,121.000,47.57,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False


In [37]:
y = train['sales']
X = train.drop(columns=['sales'])

In [38]:
del train

In [39]:
X.head()

,oil_price,transferred,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,locale_National,locale_Regional,locale_name_Cayambe,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,93.14,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [40]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [41]:
columns_to_scale = ['day_of_year', 'day_of_week']
X[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])


In [42]:
test = test.merge(oil, how="left", on="date")
test = test.merge(events, on='date', how='left')
test = test.merge(stores, on="store_nbr", how="inner")

test = pd.get_dummies(test, columns=['city', 'state'], drop_first=True)

test.replace({True: 1, False: 0}, inplace=True)
test = test.merge(transactions, on=['store_nbr', 'date'], how='left')

test['oil_price'].fillna(method='ffill', inplace=True)
test.fillna(0, inplace=True)
test = test.drop(columns=['id', 'store_nbr', 'cluster', 'type', 'onpromotion'])

test['date'] = pd.to_datetime(test['date'])
test['day_of_year'] = test['date'].dt.dayofyear
test['day_of_week'] = test['date'].dt.dayofweek
test.drop(columns=['date'], axis=1, inplace=True)

test = pd.get_dummies(test, columns=['family'], drop_first=True)

columns_to_scale = ['day_of_year', 'day_of_week']
test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])

test.replace({True: 1, False: 0}, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11388\3936124527.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['oil_price'].fillna(method='ffill', inplace=True)


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
rf_model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

In [46]:
rf_model.fit(X, y)

KeyboardInterrupt: 

In [ ]:
y_pred = rf_model.predict(test)

In [ ]:
sumb = pd.read_csv("sample_submission.csv")

In [ ]:
sumb['sales'] = y_pred

In [ ]:
sumb.to_csv("submission.csv", index=False)